In [1]:
# https://www.kaggle.com/code/sarmat/lgbm-stacking-example/notebook

In [2]:
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from dataset import custom_train_test_split, make_dataset

In [3]:
device = 'cuda'
param1 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 30,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.001
            
            }

param2 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 40,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.001
            }

param3 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 48,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.005
            }

param4 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 60,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.005
            }

param5 = {   
            'boosting': 'gbdt',
            'objective': 'binary',
            'num_leaves' : 70,
            'metric' : 'binary_logloss',
            'learning_rate' : 0.01
            }


In [4]:
cat_cols = ['i_head', 'i_mid','i_tail', 'hour', 'dow']
cont_cols = [                        
        'user_correct_answer',
        'user_total_answer',
        'user_acc',            
        't_elapsed',            
        'cum_correct',
        'last_problem',
        'head_term',
        # 'left_asymptote',
        'elo_prob',
        'pkt',
        'u_head_mean',
        'u_head_count',
        'u_head_std',
        'u_head_elapsed',
        'i_mid_elapsed',
        'i_mid_mean',
        'i_mid_std',
        'i_mid_sum',
        'i_mid_count',
        'i_mid_tag_count',
        'assessment_mean',
        'assessment_sum',
        # 'assessment_std',
        'tag_mean',
        'tag_sum',
        # 'tag_std',
        'tail_mean',
        'tail_sum',
        # 'tail_std',
        'hour_mean',
        'hour_sum',
        # 'hour_std',
        'dow_mean',
        'dow_sum',
        # 'dow_std',
        'tag_elapsed',
        'tag_elapsed_o',
        'tag_elapsed_x',
        'assessment_elapsed',
        'assessment_elapsed_o',
        'assessment_elapsed_x',
        'tail_elapsed',
        'tail_elapsed_o',
        'tail_elapsed_x']

FEATS = cat_cols + cont_cols    



In [5]:
train = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/all.pkl')
y = train.answerCode
# train, test = custom_train_test_split(train_data)
# y_train, train, y_test, test = make_dataset(train, test)
test = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/test_data.pkl')
test = test[test.answerCode==-1]

In [6]:
kf = KFold(n_splits=3, random_state = 42,shuffle = True)

oof_clf1 = np.zeros(len(train))
oof_clf2 = np.zeros(len(train))
oof_clf3 = np.zeros(len(train))
oof_clf4 = np.zeros(len(train))
oof_clf5 = np.zeros(len(train))

oof_clf1_t = np.zeros(len(test))
oof_clf2_t = np.zeros(len(test))
oof_clf3_t = np.zeros(len(test))
oof_clf4_t = np.zeros(len(test))
oof_clf5_t = np.zeros(len(test))

for (trn_idx, val_idx),test_val_idx in zip(kf.split(train.values, y.values),kf.split(test.values)):
        test_val_idx = test_val_idx[0]
        # print("fold n°{}".format(fold_))        
        
        lgb_train = lgb.Dataset(train[FEATS].iloc[trn_idx], label = y.iloc[trn_idx]) # lgb.Dataset(train[FEATS], y_train)
        lgb_valid = lgb.Dataset(train[FEATS].iloc[val_idx], label = y.iloc[val_idx]) # lgb.Dataset(valid[FEATS], y_valid)
        
        model1 = lgb.train(param1, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model2 = lgb.train(param2, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model3 = lgb.train(param3, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model4 = lgb.train(param4, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)
        model5 = lgb.train(param5, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=100,verbose_eval=100, categorical_feature = cat_cols)            
        
        oof_clf1[val_idx] = model1.predict(train[FEATS].iloc[val_idx])
        oof_clf2[val_idx] = model2.predict(train[FEATS].iloc[val_idx])
        oof_clf3[val_idx] = model3.predict(train[FEATS].iloc[val_idx])
        oof_clf4[val_idx] = model4.predict(train[FEATS].iloc[val_idx])
        oof_clf5[val_idx] = model5.predict(train[FEATS].iloc[val_idx])
        
        oof_clf1_t[test_val_idx] = model1.predict(test[FEATS].iloc[test_val_idx])
        oof_clf2_t[test_val_idx] = model2.predict(test[FEATS].iloc[test_val_idx])
        oof_clf3_t[test_val_idx] = model3.predict(test[FEATS].iloc[test_val_idx])
        oof_clf4_t[test_val_idx] = model4.predict(test[FEATS].iloc[test_val_idx])
        oof_clf5_t[test_val_idx] = model5.predict(test[FEATS].iloc[test_val_idx])
        
        # print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf1[val_idx])))
        print("AUC LGB2:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf2[val_idx])))
        print("AUC LGB3:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf3[val_idx])))
        print("AUC LGB4:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf4[val_idx])))
        print("AUC LGB5:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], oof_clf5[val_idx]))) 

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1102675, number of negative: 581295
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051052 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6624
[LightGBM] [Info] Number of data points in the train set: 1683970, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654807 -> initscore=0.640236
[LightGBM] [Info] Start training from score 0.640236
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.616366
[200]	valid_0's binary_logloss: 0.593013
[300]	valid_0's binary_logloss: 0.573518
[400]	valid_0's binary_logloss: 0.557033
[500]	valid_0's binary_logloss: 0.542937
[600]	valid_0's binary_logloss: 0.530791
[700]	valid_0's binary_logloss: 0.520344
[800]	valid_0's binary_logloss: 0.511265
[900]	valid_0's binary_logloss: 0.503352
[1000

In [ ]:
Final_train = pd.DataFrame( {
        'LGBM1': oof_clf1.ravel(),
        'LGBM2': oof_clf2.ravel(),
        'LGBM3': oof_clf3.ravel(),
        'LGBM4' :oof_clf4.ravel(),
        'LGBM5': oof_clf5.ravel(),
    })

Final_test = pd.DataFrame( {
        'LGBM1': oof_clf1_t.ravel(),
        'LGBM2': oof_clf2_t.ravel(),
        'LGBM3': oof_clf3_t.ravel(),
        'LGBM4' :oof_clf4_t.ravel(),
        'LGBM5': oof_clf5_t.ravel(),
    })

In [ ]:
param = {'objective': 'binary',
                 'boosting': 'gbdt',
                 'random_state': 42,
                 'metric': 'auc',
                 'num_threads': -1,
                 'learning_rate' : 0.1,}

In [ ]:
kf = KFold(n_splits=5, random_state = 42,shuffle = True)

Final_oof_train = np.zeros(len(train))
Final_oof_test = np.zeros(len(test))

for (trn_idx, val_idx), test_val_idx in zip(kf.split(Final_train.values, y.values), kf.split(Final_test.values)):
        # print("fold n°{}".format(fold_))
        test_val_idx = test_val_idx[0]
        lgb_train = lgb.Dataset(Final_train.iloc[trn_idx], label = y.iloc[trn_idx])
        lgb_valid = lgb.Dataset(Final_train.iloc[val_idx], label = y.iloc[val_idx])
        
        model = lgb.train(param, lgb_train, 1000, valid_sets=[lgb_valid], early_stopping_rounds=200,verbose_eval=100)
                
        Final_oof_train[val_idx] = model.predict(Final_train.iloc[val_idx])        
        
        Final_oof_test[test_val_idx] = model.predict(Final_test.iloc[test_val_idx])
        
        # print('Fold no: {}'.format(fold_))
        print("AUC LGB1:{} ".format(metrics.roc_auc_score(y.iloc[val_idx], Final_oof_train[val_idx])))

[LightGBM] [Info] Number of positive: 1323032, number of negative: 697732
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011640 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1272
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654719 -> initscore=0.639846
[LightGBM] [Info] Start training from score 0.639846
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.859143
[200]	valid_0's auc: 0.859199
[300]	valid_0's auc: 0.859203
[400]	valid_0's auc: 0.859195
Early stopping, best iteration is:
[274]	valid_0's auc: 0.859209
AUC LGB1:0.8592087895357379 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322521, number of negative: 698244
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008123 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1274
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654466 -> initscore=0.638726
[LightGBM] [Info] Start training from score 0.638726
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.859467
[200]	valid_0's auc: 0.859548
[300]	valid_0's auc: 0.859528
[400]	valid_0's auc: 0.859508
Early stopping, best iteration is:
[203]	valid_0's auc: 0.859551
AUC LGB1:0.8595507458839953 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323005, number of negative: 697760
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008914 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654705 -> initscore=0.639786
[LightGBM] [Info] Start training from score 0.639786
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.859134
[200]	valid_0's auc: 0.859175
[300]	valid_0's auc: 0.859167
Early stopping, best iteration is:
[192]	valid_0's auc: 0.859179
AUC LGB1:0.8591785036297009 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323025, number of negative: 697740
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011181 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654715 -> initscore=0.639830
[LightGBM] [Info] Start training from score 0.639830
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.859408
[200]	valid_0's auc: 0.859489
[300]	valid_0's auc: 0.859496
[400]	valid_0's auc: 0.859484
Early stopping, best iteration is:
[296]	valid_0's auc: 0.859499
AUC LGB1:0.8594987951738651 


/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322769, number of negative: 697996
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020990 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1273
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654588 -> initscore=0.639269
[LightGBM] [Info] Start training from score 0.639269
Training until validation scores don't improve for 200 rounds
[100]	valid_0's auc: 0.859492
[200]	valid_0's auc: 0.859563
[300]	valid_0's auc: 0.859566
[400]	valid_0's auc: 0.859542
Early stopping, best iteration is:
[237]	valid_0's auc: 0.85957
AUC LGB1:0.8595700525331934 


In [ ]:
Final_oof_test


0.35684508229113665

In [ ]:
def test_to_csv(preds, name:str):
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(f'output/{name}.csv', index=None)

In [ ]:
test_to_csv(Final_oof_test,'Ensemble')

NameError: name 'test_to_csv' is not defined